In [ ]:
!pip install python-dotenv

In [1]:
import os
from dotenv import load_dotenv
import concurrent
import requests
from osgeo import gdal
import lib
from importlib import reload
reload(lib)

load_dotenv()
username = os.getenv("EARTHDATA_USERNAME")
password = os.getenv("EARTHDATA_PASSWORD")
tiles_url_root = os.getenv("TILES_URL_ROOT") # The url prefix to use for the tile file names
tiles_file_path = os.getenv("TILES_FILE_PATH") # The path to a file containing a list of file names
tiles_file = open(tiles_file_path) # The file containing a list of file names
tile_file_names = tiles_file.readlines() # The list of file names

def run(sequential):
    output_path = "data/earthdata_tiles"
    os.makedirs(output_path, exist_ok=True)
    gdal.SetConfigOption('GDAL_PAM_ENABLED', 'NO')
    
    with requests.Session() as session:
        session.auth = (username, password)

        print(f'Downloading {len(tile_file_names)} files...')
        for tile_file_name in tile_file_names:
            tile_file_name = tile_file_name.strip()

            if sequential:
                lib.download_tile(
                    tiles_url_root, output_path, tile_file_name, session
                )
            else:
                with concurrent.futures.ProcessPoolExecutor() as process_pool:
                    futures = []
                    futures.append(process_pool.submit(
                        lib.download_tile,
                        tiles_url_root, output_path, tile_file_name, session
                    ))
                    for future in concurrent.futures.as_completed(futures):
                        future.result()

        print('Done!')

if __name__ == "__main__":
    run(True)


AttributeError: module 'lib' has no attribute 'download_tile'